In [1]:
import mmcv
import numpy as np
import os.path as osp
from mmdet.apis import inference_detector, init_detector

No CUDA runtime is found, using CUDA_HOME='/data/apps/CUDA/11.3.1'


/data/home/scv9654/.conda/envs/openmmlab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [ ]:
def mask_frame(img):
	frame = mmcv.imread(osp.join(CWD, 'frames', img))
	result = inference_detector(model, frame)
	mask = None
	masks = result[1][0]
	for i in range(len(masks)):
		if result[0][0][i][-1] >= 0.5:
			if not mask is None:
				mask = mask | masks[i]
			else:
				mask = masks[i]
	if mask is None:
		return
	# 获取各通道mask
	masked_b = frame[:, :, 0] * mask
	masked_g = frame[:, :, 1] * mask
	masked_r = frame[:, :, 2] * mask
	# 合并各通道
	masked = np.concatenate([masked_b[:,:,None], masked_g[:,:,None], masked_r[:,:,None]],axis=2)	
	# frame转灰度图
	un_mask = 1 - mask
	frame_b = frame[:, :, 0] * un_mask
	frame_g = frame[:, :, 1] * un_mask
	frame_r = frame[:, :, 2] * un_mask
	frame = np.concatenate([frame_b[:, :, None], frame_g[:, :, None], frame_r[:, :, None]], axis=2).astype(np.uint8)
	frame = mmcv.bgr2gray(frame, keepdim=True)
	frame = np.concatenate([frame, frame, frame], axis=2)
	# 合并
	frame += masked	
	mmcv.imwrite(frame, '../splashed_dir/' + img)	

In [2]:
# 合成视频
mmcv.frames2video('../splashed_dir/', '../out.mp4', fourcc='mp4v')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 125/125, 12.9 task/s, elapsed: 10s, ETA:     0s
